In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
import nltk
#nltk.download('stopwords')
#nltk.download('rslp') # "removedor de sufixo da lingua portuguesa"
#nltk.download('punkt') 

Auxílios do nltk usados:
    * stopwords
    * stemmer
    * wordTokenizer

In [3]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer() 


#frase = "bora ali compra um pão? bora bora!"
#x = tokenizer = nltk.tokenize.word_tokenize(frase)
#print(x)

In [4]:
classificacoes = pd.read_csv('emails.csv')
textosPuros = classificacoes['email']
#Recuperando textos em tokens e em minúsculo
frases = textosPuros.str.lower()
textosQuebrados = [nltk.tokenize.word_tokenize(frase) for frase in frases]

In [5]:
#dicionario

* Utilizando o stemmer para recupear raízes das palavras e colocar no dicionário

In [6]:
dicionario = set()
for lista in textosQuebrados:
    validas = [stemmer.stem(palavra) for palavra in lista if palavra not in stopwords and len(palavra) > 2]
    dicionario.update(validas)

In [7]:
totalDePalavras = len(dicionario)
print(f'total de palavras: {totalDePalavras}')
print(dicionario)
tuplas = zip(dicionario, range(totalDePalavras))
tradutor = {palavra: indice for palavra, indice in tuplas}

total de palavras: 230
{'bas', 'antecipad', 'equip', 'play', 'pra', 'sit', 'trilh', 'virtu', 'moder', 'exist', 'premiun', 'faculdad', 'correç', 'cust', 'sobr', 'opt', 'via', 'premium', 'víde', 'val', 'desd', 'fic', 'laravel', 'maiúscul', 'escolh', 'gost', 'ob', 'real', 'lid', 'bom', 'comput', 'sent', 'sum', 'design', 'aqu', 'qualqu', 'ganh', 'cart', 'pag', 'inic', 'iníci', 'cs', 'term', 'program', 'curs', 'cobol', 'interess', 'window', 'expir', 'troc', 'antecip', 'cri', 'coloq', 'empr', 'recomend', 'exercíci', 'carr', 'ach', 'pouc', 'cadastr', 'mim', 'emit', 'tod', 'interfac', 'almej', 'confer', 'plan', 'email', 'us', 'googl', 'respost', 'pyqt', 'framework', 'boa', 'pro', 'avis', 'depend', 'compr', 'letr', 'part', 'io', 'qual', 'moip', 'proxim', 'mud', 'depo', 'form', 'poss', 'jav', 'torn', 'sab', 'cresc', 'nov', 'quant', 'desenvolv', 'pret', 'naveg', 'ajud', 'vist', 'faz', 'vers', 'qu', 'di', 'conhec', 'áre', 'conteúd', 'pert', 'brasil', 'fórum', 'algum', 'python', 'absorv', 'complet'

In [8]:
#tradutor

* Logo, na hora de vetorizar o tezto, devo pegar as raízes das palavras contidas nele, já que o tradutor contém as raízes

In [9]:
def vetorizar_texto(texto, tradutor, stemmer):
    vetor = [0] * len(tradutor)
    for palavra in texto:
        if len(palavra) > 0:
            raiz = stemmer.stem(palavra)
            if raiz in tradutor:
                posicao = tradutor[raiz]
                vetor[posicao] += 1
    return vetor

In [10]:
vetoresDeTexto = [vetorizar_texto(texto, tradutor, stemmer) for texto in textosQuebrados]

In [11]:
marcas = classificacoes['classificacao']

X = vetoresDeTexto
Y = marcas

In [12]:
porcentagem_de_treino = 0.8

tamanho_de_treino = int(porcentagem_de_treino * len(Y))
tamanho_de_validacao = len(Y) - tamanho_de_treino

print(tamanho_de_treino)

34


In [13]:
treino_dados = X[0:tamanho_de_treino]
treino_marcacoes = Y[0:tamanho_de_treino]

validacao_dados = X[tamanho_de_treino:]
validacao_marcacoes = Y[tamanho_de_treino:]

In [14]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv=k)
    taxa_de_acerto = np.mean(scores)
    msg = "Taxa de acerto do {0}: {1}".format(nome, taxa_de_acerto)
    print(msg)
    return taxa_de_acerto

In [15]:
def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)

    acertos = resultado == validacao_marcacoes

    total_de_acertos = sum(acertos)
    total_de_elementos = len(validacao_marcacoes)

    taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

    msg = "Taxa de acerto do vencedor entre os dois algoritmos no mundo real: {0}".format(taxa_de_acerto)
    print(msg)

In [16]:
resultados = {}

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state=0))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest


from sklearn.multiclass import OneVsOneClassifier

modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state=0))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

from sklearn.naive_bayes import MultinomialNB

modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

from sklearn.ensemble import AdaBoostClassifier

modeloAdaBoost = AdaBoostClassifier(random_state=0)
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost


print(resultados)

maximo = max(resultados)
vencedor = resultados[maximo]

print("Vencerdor: ")
print(vencedor)

vencedor.fit(treino_dados, treino_marcacoes)

teste_real(vencedor, validacao_dados, validacao_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_de_acerto_base)

total_de_elementos = len(validacao_dados)
print("Total de teste: %d" % total_de_elementos)

Taxa de acerto do OneVsRest: 0.7416666666666666
Taxa de acerto do OneVsOne: 0.7416666666666667
Taxa de acerto do MultinomialNB: 0.8
Taxa de acerto do AdaBoostClassifier: 0.5833333333333333
{0.7416666666666666: OneVsRestClassifier(estimator=LinearSVC(random_state=0)), 0.7416666666666667: OneVsOneClassifier(estimator=LinearSVC(random_state=0)), 0.8: MultinomialNB(), 0.5833333333333333: AdaBoostClassifier(random_state=0)}
Vencerdor: 
MultinomialNB()
Taxa de acerto do vencedor entre os dois algoritmos no mundo real: 77.77777777777777
Taxa de acerto base: 44.444444
Total de teste: 9


APRENDIZADO:
* Stop words
* Como remover as stop words do dicionário
* Como instalar o nltk e seus módulos
* Como extrair as raízes das palavras
* Como separar as palavras de uma frase com o tokenize
* Como remover palavras com pontuações do dicionário